(method)=
# Method

* *Present the steps and components (briefly) to create an understanding of the proposed procedure. The details can come later. Highlight what makes it unique compared to other studies in the literature (include references).*

## Overview of the proposed procedure

[Fig](overview) shows a flow chart summarizing how the PIT works.
```{figure} figures/method.png
---
width: 500px
name: overview
---
Flow chart over the proposed Parameter Identification Technique (PIT)
```

### VMM
The VMM (see section  [VMM](vmm) ) expresses the dynamics of a ship. This ability is most often used to conduct manoeuvring simulations, where a VMM with known forces from hydrodynamic derivatives is used to predict unknown ship manoeuvres. 

### Inverse dynamics
VMM can also be used to solve the reversed problem: predicting unknown forces from known ship manoeuvres, which is applicable for the current problem. This is called inverse dynamics (see section [Inverse-dynamics](inverse_dynamics)). 

### Regression
The hydrodynamic derivatives in the VMM can be identified with regression of the force polynomials on forces predicted with inverse dynamics, which is called ''motion regression'' in this paper. Many authors have used different statistical methods or machine learning methods to regress the hydrodynamic derivatives. The Linear regression with Ordinary Least Square fit (OLS) is used in the proposed PIT. The OLS is known to be sensitive to noise and outliers, which is why the main focus in the present PIT is on pre-processing data with filtering rather than the regression method itself.

### EKF
The filtering in the PIT is conducted with an Extended Kalman Filter (EKF) (see section [EKF](EKF)) which is a recursive filter that estimates the state of a linear or nonlinear dynamic system from a series of noisy measurements. The basic idea is that noise can be disregarded if it does not make sense from a physical point of view. If noisy measurement data would be perfectly correct, this would mean that the ship has a lot of vibrations that must have originated from huge forces, considering the large mass of the ship. The prior understanding of the model tests data suggests that these huge forces are not present during the model test, and the noise is therefore considered as measurement noise that should be removed. Lowpass filtering is a common way to do this, where motions above some cut-off frequency are regarded as unphysical measurement noise. The problem with lowpass filter is that it is hard to know what the cut-off frequency to choose, either too low: removing part of the signal, or too high: keeping some unfiltered measurement noise in the data. 
The Kalman filter has a system model that continuously estimates the state of the system that is run in parallel with the measurement data. The filter estimates the current state as a combination of the measurement data and the system model estimate based on belief in the data and the model. If the data has low noise the estimate turns towards that data, if on the other hand the model gives very good predictions that estimate turns towards the model. The inverse dynamics requires the entire state of the system to be known including: positions, velocities and accelerations. Only positions are known from the measurements which means that velocities and accelerations are hidden states that can be estimated by EKF.

### RTS
The EKF is recursive and can be run online, continuously making new estimates as new measurements arrive. It uses passed measurements to estimate states in the near future. This is useful for applications like autopilots. For the PIT the filter is instead run on a whole time series of existing measurements. The fact that both past and future data is known can be used to improve the filter by attaching a smoother after the EKF.  The PIT uses a Rauch Tung Striebel (RTS) smoother (see section [RTS](RTS)). 


A VMM is used as the system model in the EKF. But the identified VMM is of course not known yet which is a ''the chicken or the egg'' situation. This is solved with initial guessing and iteration.

### Step 1) 
VMM with guessed hydrodynamic derivatives is used. To make a fair guess, the derivatives are estimated with semiempirical formulas for a linear VMM. The VMM is used in the EKF and RTS smoother to filter all the model tests. The VMM:s are assumed to have Markov property which means that future states depend only on the current state. This means that the filtered data with estimated hidden states from all the model tests can be joined into one time independent dataset that is passed to the motion regression. The hydrodynamic derivatives are regressed on quasi static forces from inverse dynamics giving the identified nonlinear VMM.

### Step 2) 
Step1 can now be rerun, but instead of using the guessed system model, the identified VMM from step 1 can now be used in the EKF. There should be a higher belief in this model than the guessed model, so the covariance matrixes should be updated.  

![](figures/method.png)